In [1]:
import dotenv
dotenv.load_dotenv()
import os
print(os.getenv("TOGETHER_API_KEY"))

5a9bb14c549b3015696e7dc92d8382b073cf918e119110c61374fcd5be4ee219


In [2]:
import dspy

llama3b = dspy.LM('together_ai/meta-llama/Llama-3.2-3B-Instruct-Turbo', temperature=0.7, api_key = os.getenv("TOGETHER_API_KEY"))
gpt4o = dspy.LM('openai/gpt-4o', temperature=0.7)

dspy.configure(lm=llama3b)

In [3]:
import random
from dspy.datasets import DataLoader

kwargs = dict(fields=("claim", "supporting_facts", "hpqa_id", "num_hops"), input_keys=("claim",))
hover = DataLoader().from_huggingface(dataset_name="hover-nlp/hover", split="train", trust_remote_code=True, **kwargs)

hpqa_ids = set()
hover = [
    dspy.Example(claim=x.claim, titles=list(set([y["key"] for y in x.supporting_facts]))).with_inputs("claim")
    for x in hover
    if x["num_hops"] == 3 and x["hpqa_id"] not in hpqa_ids and not hpqa_ids.add(x["hpqa_id"])
]

random.Random(0).shuffle(hover)
trainset, devset, testset = hover[:100], hover[100:200], hover[650:]

/Users/tytodd/Library/Caches/pypoetry/virtualenvs/modaic-cVI88qFt-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
example = trainset[0]

print("Claim:", example.claim)
print("Pages that must be retrieved:", example.titles)

Claim: This director is known for his work on Miss Potter. The Academy of Motion Picture Arts and Sciences presents the award in which he was nominated for his work in "Babe".
Pages that must be retrieved: ['Chris Noonan', 'Academy Award for Best Director', 'Miss Potter']


In [5]:
DOCS = {}

def search(query: str, k: int) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=k)
    results = [x['text'] for x in results]

    for result in results:
        title, text = result.split(" | ", 1)
        DOCS[title] = text

    return results

In [6]:
def search_wikipedia(query: str) -> list[str]:
    """Returns top-5 results and then the titles of the top-5 to top-30 results."""

    topK = search(query, 30)
    titles, topK = [f"`{x.split(' | ')[0]}`" for x in topK[5:30]], topK[:5]
    return topK + [f"Other retrieved pages have titles: {', '.join(titles)}."]

def lookup_wikipedia(title: str) -> str:
    """Returns the text of the Wikipedia page, if it exists."""

    if title in DOCS:
        return DOCS[title]

    results = [x for x in search(title, 10) if x.startswith(title + " | ")]
    if not results:
        return f"No Wikipedia page found for title: {title}"
    return results[0]

In [7]:
instructions = "Find all Wikipedia titles relevant to verifying (or refuting) the claim."
signature = dspy.Signature("claim -> titles: list[str]", instructions)
react = dspy.ReAct(signature, tools=[search_wikipedia, lookup_wikipedia], max_iters=20)

In [8]:
react(claim="David Gregory was born in 1625.").titles[:3]

['David Gregory', 'David Gregory (1625)', 'David Gregory (no year)']

In [9]:
def top5_recall(example, pred, trace=None):
    gold_titles = example.titles
    recall = sum(x in pred.titles[:5] for x in gold_titles) / len(gold_titles)

    # If we're "bootstrapping" for optimization, return True if and only if the recall is perfect.
    if trace is not None:
        return recall >= 1.0
    
    # If we're just doing inference, just measure the recall.
    return recall

evaluate = dspy.Evaluate(devset=devset, metric=top5_recall, num_threads=16, display_progress=True, display_table=5)

In [10]:
def safe_react(claim: str):
    try:
        return react(claim=claim)
    except Exception as e:
        return dspy.Prediction(titles=[])

evaluate(safe_react)

Average Metric: 0.67 / 58 (1.1%):  58%|█████▊    | 58/100 [02:21<02:43,  3.89s/it]

2025/07/19 17:00:28 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.7)  if the reason for truncation is repetition.


Average Metric: 0.67 / 85 (0.8%):  85%|████████▌ | 85/100 [03:18<00:35,  2.34s/it]

2025/07/19 17:01:30 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 3.00 / 100 (3.0%): 100%|██████████| 100/100 [04:33<00:00,  2.73s/it]

2025/07/19 17:02:39 INFO dspy.evaluate.evaluate: Average Metric: 2.9999999999999996 / 100 (3.0%)


,claim,example_titles,pred_titles,top5_recall,trajectory,reasoning
0,The Church of England's movement that inspired the Trinity Episcop...,"[Trinity Episcopal Church (Houghton, Michigan), Samuel Rickards, O...",[],,NaN,NaN
1,"Red, White & Crüe and this athlete both fight. The french fighter ...","[Mike Tyson, Bobby Stewart, Red, White &amp; Crüe]",[],,NaN,NaN
2,The writer/director/actor from Glen or Glenda and Fernand Rivers s...,"[Ed Wood, Glen or Glenda, Fernand Rivers]",[],,NaN,NaN
3,The film by Sandi Sissel was released before The End of Suburbia.,"[Chicken Ranch (film), The End of Suburbia, Sandi Sissel]",[],,NaN,NaN
4,The actor who played captain hook in the live production with Tayl...,"[Christopher Walken, Peter Pan Live!, Taylor Louderman]",[],,NaN,NaN


3.0

In [13]:
kwargs = dict(teacher_settings=dict(lm=gpt4o), prompt_model=gpt4o, max_errors=999)

tp = dspy.MIPROv2(metric=top5_recall, auto="medium", num_threads=16, **kwargs)
optimized_react = tp.compile(react, trainset=trainset, max_bootstrapped_demos=3, max_labeled_demos=0)

2025/07/19 17:02:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 28
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 80



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 6 * 2 lm calls in program + (3) lm calls in program-aware proposer = 25 prompt model calls
- Program Evaluation: 35 examples in minibatch * 28 batches + 80 examples in val set * 6 full evals = 1460 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token)
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectati

2025/07/19 17:02:59 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/07/19 17:02:59 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/07/19 17:02:59 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...



No input received within 20 seconds. Proceeding with execution...
Bootstrapping set 1/12
Bootstrapping set 2/12


 35%|███▌      | 7/20 [00:52<01:37,  7.48s/it]


...

Average Metric: 2.00 / 7 (28.6%):  63%|██████▎   | 22/35 [00:45<00:19,  1.48s/it]

2025/07/19 17:53:03 ERROR dspy.utils.parallelizer: Error for Example({'claim': 'Fredric March costarred in 1935 Australian drama film that Frances Drake is best know for, with the husband of Elsa Lanchester.', 'titles': ['Frances Drake', 'Charles Laughton', 'Les Misérables (1935 film)']}) (input_keys={'claim'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: RateLimitError: Together_aiException - Request was rejected due to request rate limiting. Your rate limits are 60 RPM (1 QPS) and 60000 TPM (1000 TPS). See details: https://docs.together.ai/docs/rate-limits. Set `provide_traceback=True` for traceback.


Average Metric: 2.00 / 8 (25.0%):  69%|██████▊   | 24/35 [00:51<00:21,  1.99s/it]

2025/07/19 17:53:05 ERROR dspy.utils.parallelizer: Error for Example({'claim': 'The Stanley Kubrick thriller that Nicole Kidman worked with Tom Cruise on, starring Vinessa Elizabeth Shaw, was based on the novella "Traumnovelle" ("Dream Story") .', 'titles': ['Vinessa Shaw', 'Eyes Wide Shut', 'Nicole Kidman filmography']}) (input_keys={'claim'}): litellm.RateLimitError: RateLimitError: Together_aiException - Request was rejected due to request rate limiting. Your rate limits are 60 RPM (1 QPS) and 60000 TPM (1000 TPS). See details: https://docs.together.ai/docs/rate-limits. Set `provide_traceback=True` for traceback.


Average Metric: 2.00 / 8 (25.0%):  71%|███████▏  | 25/35 [00:53<00:18,  1.89s/it]

In [ ]:
evaluate(optimized_react)

In [ ]:
optimized_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles

In [ ]:
optimized_react.save("optimized_react.json")

loaded_react = dspy.ReAct("claim -> titles: list[str]", tools=[search_wikipedia, lookup_wikipedia], max_iters=20)
loaded_react.load("optimized_react.json")

loaded_react(claim="The author of the 1960s unproduced script written for The Beatles, Up Against It, and Bernard-Marie Koltès are both playwrights.").titles